In [ ]:
!pip install -q wfdb torchinfo mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 141.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 154.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [ ]:
import os
import math
import random
from collections import defaultdict, Counter
from typing import List, Optional

import h5py
import numpy as np
import pandas as pd
import pywt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.pytorch
import wfdb
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/oladipoeyiara@gmail.com/han_cinc_full")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3223963235727852', creation_time=1768068298180, experiment_id='3223963235727852', last_update_time=1768073253610, lifecycle_stage='active', name='/Users/oladipoeyiara@gmail.com/han_cinc_full', tags={'mlflow.experiment.sourceName': '/Users/oladipoeyiara@gmail.com/han_cinc_full',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'oladipoeyiara@gmail.com',
 'mlflow.ownerId': '1108839756692281'}>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/CINC2020.zip'
extract_dir = '/content/'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
# -------------------------
# Config
# -------------------------
SAMPLING_RATE = 500
PRE_PEAK_SAMPLES = 99
POST_PEAK_SAMPLES = 201
SEGMENT_LENGTH = PRE_PEAK_SAMPLES + POST_PEAK_SAMPLES  # 300
DROP_LAST = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# -------------------------
# utilities: filtering and peak detection
# -------------------------
def denoise(data):
    # wavelet transform
    coeffs = pywt.wavedec(data=data, wavelet='db5', level=9)
    cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs

    # Threshold denoising
    threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
    cD1.fill(0)
    cD2.fill(0)
    for i in range(1, len(coeffs) - 2):
        coeffs[i] = pywt.threshold(coeffs[i], threshold)

    # Inverse wavelet transform to obtain the denoised signal
    rdata = pywt.waverec(coeffs=coeffs, wavelet='db5')
    return rdata


def pan_tompkins_detector(ecg_signal, fs):
    lowcut, highcut = 5.0, 15.0
    nyquist = 0.5 * fs
    low, high = lowcut / nyquist, highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_ecg = filtfilt(b, a, ecg_signal)
    diff_ecg = np.diff(filtered_ecg)
    squared_ecg = diff_ecg ** 2
    window_size = int(0.150 * fs)
    mwa_ecg = np.convolve(squared_ecg, np.ones(window_size) / window_size, mode='same')
    peaks, _ = find_peaks(mwa_ecg, distance=int(0.6 * fs))
    return peaks


def multi_lead_fusion(detected_peaks, fs, fusion_window=0.1, min_leads=None):
    n_leads = len(detected_peaks)
    if min_leads is None:
        min_leads = int(np.ceil(n_leads / 2))

    # Collect all peaks with their lead information
    all_peaks = [(p, lead) for lead, peaks in enumerate(detected_peaks) for p in peaks]
    all_peaks.sort(key=lambda x: x[0])

    fused_peaks = []
    i = 0

    while i < len(all_peaks):
        # Start a new cluster
        cluster = [all_peaks[i]]
        i += 1

        # Add nearby peaks to the cluster
        while i < len(all_peaks) and all_peaks[i][0] - cluster[-1][0] <= fusion_window * fs:
            cluster.append(all_peaks[i])
            i += 1

        # Check if cluster has peaks from enough leads
        unique_leads = {lead for _, lead in cluster}
        if len(unique_leads) >= min_leads:
            # Use median position as the fused peak
            fused_peak = int(np.median([p for (p, _) in cluster]))
            fused_peaks.append(fused_peak)

    return np.array(sorted(fused_peaks))


def detect_r_peaks(ecg_signals, fs):
    detected_peaks = []
    for lead in ecg_signals:
        peaks = pan_tompkins_detector(lead, fs)
        detected_peaks.append(peaks)

    fused_r_peaks = multi_lead_fusion(detected_peaks, fs, fusion_window=0.1, min_leads=6)
    return fused_r_peaks


def extract_segments_around_peaks(signal, r_peaks, pre_samples, post_samples):
    segments = []

    for peak in r_peaks:
        start = max(0, peak - pre_samples)
        end = min(len(signal), peak + post_samples)

        # Only include segments with the correct length
        if end - start == pre_samples + post_samples:
            segment = signal[start:end]
            segments.append(segment)

    return segments


def extract_beats_multi_lead(ecg_signals, fs, pre_samples, post_samples, denoise_fn=None):
    # Convert to numpy array for consistent indexing
    ecg_signals = np.array(ecg_signals)

    # Apply denoising if provided
    if denoise_fn is not None:
        ecg_signals = np.array([denoise_fn(lead) for lead in ecg_signals])

    # Detect fused R-peaks across all leads
    r_peaks = detect_r_peaks(ecg_signals, fs)
    # print(f"Detected {len(r_peaks)} R-peaks after fusion")

    if len(r_peaks) == 0:
        return None

    # Extract segments from all leads
    all_lead_segments = []
    for lead_idx, lead_signal in enumerate(ecg_signals):
        segments = extract_segments_around_peaks(lead_signal, r_peaks, pre_samples, post_samples)
        all_lead_segments.append(segments)

    # Find the minimum number of valid segments across all leads
    min_segments = min(len(segments) for segments in all_lead_segments)
    # print(f"Extracted {min_segments} valid segments across all leads")

    if min_segments == 0:
        return None

    # Stack segments: (n_beats, segment_length, n_leads)
    beats_arr = np.stack([
        np.stack(segments[:min_segments], axis=0)
        for segments in all_lead_segments
    ], axis=-1)

    return beats_arr.astype(np.float32)

In [ ]:
print("Loading and preprocessing ECG data...")

RECORDS = "classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/RECORDS"
LEAD_TO_KEEP = 1

available_labels = ['10370003', '164889003', '164909002', '164934002', '270492004', '284470004', '426177001', '426783006', '427084000', '427393009', '59118001']

tmp = []  # temporarily store (beats_array, label_str)

with open(f"{RECORDS}") as f:
    folders_list = f.read()
    FOLDERS = folders_list.strip().splitlines()

for folder in FOLDERS:
    if folder in ["training/ptb/g1/", "training/st_petersburg_incart/g1/"]:
        continue

    path = f"classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/{folder}/RECORDS"
    with open(path) as r:
        data_files = r.read()
        files = data_files.strip().splitlines()[:-1]

        for file_name in files:
            final_data_path = f"classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/{folder}/{file_name}"
            try:
                hdr = wfdb.rdheader(final_data_path)
            except Exception as e:
                print("error reading header", final_data_path, e)
                continue

            # find a single Dx: comment (skip multi-label/comment with commas)
            label = None
            for comment in hdr.comments:
                if comment.startswith("Dx:") and "," not in comment:
                    label = comment.replace("Dx: ", "")
                    break

            if label is None or label not in available_labels:
                continue

            try:
                record = wfdb.rdrecord(final_data_path)
                signal = record.p_signal
            except Exception as e:
                print("error reading record", final_data_path, e)
                continue

            # Replace NaNs
            if np.isnan(signal).any():
                signal = np.nan_to_num(signal, nan=0.0)

            # Shape should be (12 (leads), SEQ_LEN)
            signal = signal.T

            # Denoise the selected lead
            denoised = np.array([denoise(lead) for lead in signal])

            denoised = np.nan_to_num(signal, nan=0.0)

            if np.isnan(denoised).any():
                print("-------------")
                print("WARNING: denoised signal contains NaNs for", final_data_path)

            record_segment = denoised

            # extract beats from this single-lead segment
            beats = extract_beats_multi_lead(
                record_segment,
                fs=SAMPLING_RATE,
                pre_samples=PRE_PEAK_SAMPLES,
                post_samples=POST_PEAK_SAMPLES,
                denoise_fn=None  # already denoised
            )

            if beats is None or getattr(beats, "shape", (0,))[0] == 0:
                # no usable beats -> skip this record
                continue

            tmp.append((beats.astype(np.float32), label))

# build label -> index mapping from labels actually present
unique_labels = sorted({lbl for (_, lbl) in tmp})
label2idx = {lbl: i for i, lbl in enumerate(unique_labels)}
num_classes = len(unique_labels)
print("num_classes:", num_classes, "labels:", unique_labels)

# convert to final data_list with integer labels
data_list = []
for beats, lbl in tmp:
    if lbl not in label2idx:
        continue
    data_list.append((beats, label2idx[lbl]))

from collections import Counter
print("Prepared records:", len(data_list))
print("Beat counts distribution:", Counter([x[0].shape[0] for x in data_list]))


Loading and preprocessing ECG data...


/usr/local/lib/python3.12/dist-packages/pywt/_thresholding.py:22: RuntimeWarning: invalid value encountered in divide
  thresholded = (1 - value/magnitude)
/usr/local/lib/python3.12/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 9 is too high: all coefficients will experience boundary effects.
  warnings.warn(


num_classes: 11 labels: ['10370003', '164889003', '164909002', '164934002', '270492004', '284470004', '426177001', '426783006', '427084000', '427393009', '59118001']
Prepared records: 14529
Beat counts distribution: Counter({11: 2727, 10: 2406, 12: 2189, 13: 1557, 9: 1456, 14: 1094, 15: 686, 8: 373, 16: 263, 17: 191, 18: 181, 19: 133, 20: 127, 21: 101, 22: 89, 23: 71, 24: 69, 26: 58, 25: 52, 31: 45, 28: 44, 29: 43, 27: 40, 30: 38, 33: 29, 39: 26, 35: 25, 32: 23, 40: 22, 38: 22, 34: 21, 37: 21, 41: 19, 43: 19, 45: 19, 42: 17, 46: 17, 36: 16, 47: 13, 58: 11, 6: 11, 44: 11, 49: 10, 55: 10, 48: 10, 52: 9, 53: 9, 51: 9, 50: 8, 54: 7, 5: 7, 57: 6, 74: 5, 56: 4, 61: 4, 72: 4, 7: 4, 78: 3, 67: 3, 62: 3, 4: 3, 59: 2, 77: 2, 68: 2, 87: 2, 63: 2, 60: 2, 101: 2, 64: 2, 75: 2, 84: 2, 70: 2, 65: 1, 80: 1, 76: 1, 86: 1, 82: 1, 2: 1, 85: 1, 151: 1, 66: 1, 91: 1, 92: 1, 79: 1, 95: 1, 93: 1})


In [ ]:
data_list[3][0].shape

(14, 300, 12)

In [ ]:
# ---------- Length-bucketing Batch Sampler ----------
class LengthBucketBatchSampler(Sampler):
    """
    Yields lists of indices (batches) where all samples in a batch share the same length-bucket.

    Args:
        lengths: list/array-like of int lengths (len(dataset) entries).
        batch_size: target batch size.
        bin_size: None for exact-length buckets. If int > 0, length_key = (length // bin_size) * bin_size.
        shuffle: shuffle within buckets and shuffle batch order each epoch.
        drop_last: whether to drop the last small batch in a bucket.
    """
    def __init__(self,
                 lengths: List[int],
                 batch_size: int,
                 bin_size: Optional[int] = None,
                 shuffle: bool = True,
                 drop_last: bool = False):
        self.lengths = list(lengths)
        self.batch_size = int(batch_size)
        self.bin_size = bin_size
        self.shuffle = shuffle
        self.drop_last = drop_last

        # Build mapping length_key -> list of indices
        self._buckets = defaultdict(list)
        for idx, L in enumerate(self.lengths):
            key = self._length_key(L)
            self._buckets[key].append(idx)

        # Convert to normal dict for iteration; keep keys list stable
        self.bucket_keys = list(self._buckets.keys())

    def _length_key(self, length: int) -> int:
        if self.bin_size is None or self.bin_size <= 0:
            return int(length)   # exact-length bucket
        else:
            return (length // self.bin_size) * self.bin_size

    def __iter__(self):
      # For each epoch, build batches from buckets.
      batches = []
      for key in self.bucket_keys:
          idxs = list(self._buckets[key])
          if len(idxs) < self.batch_size:
              # skip this bucket entirely
              continue
          if self.shuffle:
              random.shuffle(idxs)
          # chunk into batches
          for i in range(0, len(idxs), self.batch_size):
              batch = idxs[i:i + self.batch_size]
              if len(batch) < self.batch_size and self.drop_last:
                  continue
              batches.append(batch)

      if self.shuffle:
          random.shuffle(batches)

      for batch in batches:
          yield batch

    def __len__(self):
        total = 0
        for key in self.bucket_keys:
            n = len(self._buckets[key])
            if self.drop_last:
                total += n // self.batch_size
            else:
                total += math.ceil(n / self.batch_size)
        return total

In [ ]:
# -------------------------
# Dataset + Sampler + DataLoader
# -------------------------
class ECGSegmentDataset(Dataset):
    def __init__(self, data_list):
        # data_list: list of (beats_array (S, T, C), label_idx)
        self.data = data_list
        self.num_beats = [int(x[0].shape[0]) for x in data_list]
        # optionally check that T==SEGMENT_LENGTH for all entries
        for sig, _ in data_list:
            assert sig.shape[1] == SEGMENT_LENGTH, "SEGMENT_LENGTH mismatch"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sig, label = self.data[idx]
        sig_t = torch.as_tensor(sig, dtype=torch.float32)  # (S, T, C)
        return {"signal": sig_t, "label": int(label), "num_beats": int(sig_t.shape[0])}


def collate_by_num_beats(batch):
    s_vals = [item["num_beats"] for item in batch]
    if not all(s == s_vals[0] for s in s_vals):
        raise ValueError("collate_by_num_beats received mixed num_beats in a batch")
    signals = torch.stack([item["signal"] for item in batch], dim=0)  # (B, S, T, C)
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.long)
    return {"signal": signals, "label": labels, "num_beats": torch.tensor(s_vals, dtype=torch.long)}


In [ ]:
from torch.utils.data import Subset, DataLoader

# # instantiate dataset and sampler
ds = ECGSegmentDataset(data_list)

# -------------------------
# Split indices
# -------------------------
all_indices = list(range(len(ds)))
all_labels = [data_list[i][1] for i in all_indices]

# First split: train+val vs test (stratified)
trainval_indices, test_indices = train_test_split(
    all_indices,
    test_size=0.1,
    random_state=10,
    stratify=all_labels  # ← stratify by class labels
)

# Extract labels for the trainval subset
trainval_labels = [all_labels[i] for i in trainval_indices]

# Second split: train vs val (stratified)
train_indices, val_indices = train_test_split(
    trainval_indices,
    test_size=0.1,
    random_state=10,
    stratify=trainval_labels  # ← stratify by class labels
)

# -------------------------
# Create Subsets
# -------------------------
train_ds = Subset(ds, train_indices)
val_ds   = Subset(ds, val_indices)
test_ds  = Subset(ds, test_indices)

# -------------------------
# Make loader helper
# -------------------------
def make_loader(subset_ds, batch_size=16, drop_last=True, shuffle=True):
    # subset_ds is a Subset, so subset_ds.indices are the original dataset indices
    subset_indices = subset_ds.indices
    # use the original dataset's num_beats
    lengths = [ds.num_beats[i] for i in subset_indices]

    sampler = LengthBucketBatchSampler(
        lengths=lengths,
        batch_size=batch_size,
        bin_size=None,       # exact-length buckets
        shuffle=shuffle,
        drop_last=drop_last
    )

    loader = DataLoader(
        subset_ds,
        batch_sampler=sampler,
        collate_fn=collate_by_num_beats,
        num_workers=6,
        pin_memory=True
    )
    return loader

In [ ]:
BATCH_SIZE=16

# Verifying stratifying
train_loader = make_loader(train_ds, batch_size=BATCH_SIZE)
val_loader   = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = make_loader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

def get_label_distribution(indices):
    labels = [data_list[i][1] for i in indices]
    return Counter(labels)

print("Train distribution:", get_label_distribution(train_indices))
print("Val distribution:", get_label_distribution(val_indices))
print("Test distribution:", get_label_distribution(test_indices))

Train distribution: Counter({7: 7363, 10: 1299, 1: 879, 4: 610, 5: 436, 6: 369, 0: 220, 8: 185, 2: 175, 3: 129, 9: 103})
Val distribution: Counter({7: 818, 10: 144, 1: 98, 4: 68, 5: 48, 6: 41, 0: 25, 8: 21, 2: 20, 3: 14, 9: 11})
Test distribution: Counter({7: 909, 10: 160, 1: 108, 4: 75, 5: 54, 6: 46, 0: 27, 8: 23, 2: 22, 3: 16, 9: 13})


In [ ]:
# -------------------------
# Attention / helper layers
# -------------------------
class ChannelAttention(nn.Module):
    """
    Expects x shape = (batch, channels, seq_len)
    """
    def __init__(self, channels, ratio=8):
        super().__init__()
        mid = max(1, channels // ratio)
        self.mlp = nn.Sequential(
            nn.Linear(channels, mid, bias=True),
            nn.ReLU(),
            nn.Linear(mid, channels, bias=True)
        )

    def forward(self, x):
        # x: (B, C, L)
        avg_pool = torch.mean(x, dim=2)           # (B, C)
        max_pool, _ = torch.max(x, dim=2)         # (B, C)
        avg_out = self.mlp(avg_pool)              # (B, C)
        max_out = self.mlp(max_pool)              # (B, C)
        att = torch.sigmoid(avg_out + max_out)    # (B, C)
        att = att.unsqueeze(2)                    # (B, C, 1)
        return x * att                             # broadcast multiply -> (B, C, L)


class SegmentAttention(nn.Module):
    """
    Input: (batch, time_steps, input_dim)
    Produces: (batch, input_dim), (batch, time_steps)  (output vector, alphas)
    `units` is the hidden size of the attention MLP (same as original design).
    """
    def __init__(self, input_dim, units):
        super().__init__()
        self.linear = nn.Linear(input_dim, units, bias=True)  # W and b equivalent
        # u vector for scoring
        self.u = nn.Parameter(torch.randn(units))

    def forward(self, inputs):
        # inputs: (B, T, D)
        v = torch.tanh(self.linear(inputs))           # (B, T, units)
        # score each time step
        vu = torch.matmul(v, self.u)                  # (B, T)
        alphas = F.softmax(vu, dim=1)                 # softmax over time dimension
        output = torch.sum(inputs * alphas.unsqueeze(-1), dim=1)  # (B, D)
        return output, alphas


class TimeDistributedSegmentAttention(nn.Module):
    """
    inputs: (B, segments, seg_len, feature_dim)
    returns: (B, segments, feature_dim), (B, segments, seg_len)
    """
    def __init__(self, input_dim, units):
        super().__init__()
        self.segment_attention = SegmentAttention(input_dim, units)

    def forward(self, inputs):
        # inputs: (B, S, T, D)
        B, S, T, D = inputs.shape
        flat = inputs.view(B * S, T, D)                       # (B*S, T, D)
        outputs, alphas = self.segment_attention(flat)        # (B*S, D), (B*S, T)
        outputs = outputs.view(B, S, D)                       # (B, S, D)
        alphas = alphas.view(B, S, T)                         # (B, S, T)
        return outputs, alphas


class HANWithAttention(nn.Module):
    def __init__(self, num_classes=11):
        super().__init__()

        # Conv1d: in_channels=12, out_channels=128, kernel_size=25, padding=12
        self.conv1d = nn.Conv1d(in_channels=12, out_channels=128, kernel_size=25, padding=12)

        # Channel attention on 128 channels with ratio 8
        self.channel_attention = ChannelAttention(128, ratio=8)

        # MaxPool1d kernel 3, stride 2, padding 1
        self.pool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        # Segment-level LSTM: input_size=128, hidden_size=64
        self.lstm_segment = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)

        # TimeDistributed segment attention: input_dim=64, units=64
        self.time_distributed_attention = TimeDistributedSegmentAttention(input_dim=256, units=256)

        # Sequence-level LSTM over segments: input_size=64, hidden_size=64
        self.lstm_sequence = nn.LSTM(input_size=256, hidden_size=256, batch_first=True)

        # Final attention across segments: input_dim=64, units=64
        self.final_attention = SegmentAttention(input_dim=256, units=256)

        # Classifier head
        self.fc = nn.Linear(256, 512)
        self.dropout = nn.Dropout(0.4)
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        logits, _, _ = self.forward_with_attention(x)
        return logits

    def forward_with_attention(self, x):
        B, S, T, C = x.shape
        x = x.view(B * S, T, C).permute(0, 2, 1)  # (B*S, C, T)

        conv = self.conv1d(x)                     # (B*S, 128, T)
        att = self.channel_attention(conv)        # (B*S, 128, T)
        pooled = self.pool(att)                   # (B*S, 128, T2)
        pooled = pooled.permute(0, 2, 1)          # (B*S, T2, 128)

        seg_lstm_out, _ = self.lstm_segment(pooled)  # (B*S, T2, 64)
        seg_lstm_out = seg_lstm_out.view(B, S, seg_lstm_out.shape[1], seg_lstm_out.shape[2])  # (B, S, T2, 64)

        segment_outputs, segment_alphas = self.time_distributed_attention(seg_lstm_out)       # (B, S, 64), (B, S, T2)
        seq_lstm_out, _ = self.lstm_sequence(segment_outputs)                                  # (B, S, 64)
        final_output, final_alphas = self.final_attention(seq_lstm_out)                        # (B, 64), (B, S)

        x = F.relu(self.fc(final_output))
        x = self.dropout(x)
        logits = self.classifier(x)  # (B, num_classes)

        return logits, final_alphas, segment_alphas

    def get_attention_weights(self, x):
        _, final_alphas, segment_alphas = self.forward_with_attention(x)
        return final_alphas, segment_alphas

from torchinfo import summary

model = HANWithAttention(num_classes=11)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HANWithAttention().to(device)

summary(model,
        input_size=(2, 10, 300, 12),    # (batch, segments, timesteps, channels)
        col_names=("input_size", "output_size", "num_params", "trainable"),
        depth=4,
        device=device.type)           



Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
HANWithAttention                              [2, 10, 300, 12]          [2, 11]                   --                        True
├─Conv1d: 1-1                                 [20, 12, 300]             [20, 128, 300]            38,528                    True
├─ChannelAttention: 1-2                       [20, 128, 300]            [20, 128, 300]            --                        True
│    └─Sequential: 2-1                        [20, 128]                 [20, 128]                 --                        True
│    │    └─Linear: 3-1                       [20, 128]                 [20, 16]                  2,064                     True
│    │    └─ReLU: 3-2                         [20, 16]                  [20, 16]                  --                        --
│    │    └─Linear: 3-3                       [20, 16]                  [20, 128]             

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

from tqdm import tqdm

def evaluate_metrics(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for batch in loader:
            inputs = batch["signal"].to(device)
            labels = batch["label"].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            preds = outputs.argmax(dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            all_labels.extend(labels)
            all_preds.extend(preds)
            all_probs.extend(probs)

    avg_loss = running_loss / len(all_labels)
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

    try:
      y_true = np.eye(num_classes)[all_labels]
      y_score = np.array(all_probs)

      auc_list = []
      for i in range(num_classes):
          if np.any(y_true[:, i]):  # class i exists
              auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
      if auc_list:
          auc = np.mean(auc_list)
      else:
          auc = float("nan")
    except ValueError:
        auc = float("nan")

    return avg_loss, acc, precision, recall, auc, f1

def train(model, train_loader, val_loader, optimizer, criterion, device,
          epochs, scheduler=None):
    model.to(device)
    history = {
        "train_loss": [], "train_acc": [], "train_f1": [], "train_precision": [], "train_recall": [], "train_auc": [],
        "val_loss": [],   "val_acc": [],   "val_f1": [],   "val_precision": [],   "val_recall": [], "val_auc": []
    }

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        all_preds = []
        all_probs = []
        all_labels = []
        steps = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
        for batch in loop:
            inputs = batch["signal"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            batch_size = labels.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim=1)
            probs = torch.softmax(outputs, dim=1).detach().cpu().numpy()
            correct += (preds == labels).sum().item()
            total += batch_size

            all_preds.extend(preds.cpu().numpy().tolist())
            all_probs.extend(probs.tolist())
            all_labels.extend(labels.cpu().numpy().tolist())

            steps += 1
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{(correct/total):.4f}")

        if scheduler is not None:
            try:
                scheduler.step()
            except Exception:
                pass

        # if statement is kinda useless
        if total == 0:
            epoch_loss = float("nan")
            epoch_acc = float("nan")
            epoch_precision = epoch_recall = epoch_f1 = float("nan")
        else:
            epoch_loss = running_loss / total
            epoch_acc = correct / total
            epoch_precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
            epoch_recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
            epoch_f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)
            try:
              y_true = np.eye(num_classes)[all_labels]
              y_score = np.array(all_probs)

              auc_list = []
              for i in range(num_classes):
                  if np.any(y_true[:, i]):
                      auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
              if auc_list:
                  auc = np.mean(auc_list)
              else:
                  auc = float("nan")
            except ValueError:
                auc = float("nan")

        # append train history
        history["train_loss"].append(epoch_loss)
        history["train_auc"].append(auc)
        history["train_acc"].append(epoch_acc)
        history["train_precision"].append(epoch_precision)
        history["train_recall"].append(epoch_recall)
        history["train_f1"].append(epoch_f1)

        print(f"Epoch {epoch}/{epochs}  train_loss: {epoch_loss:.4f}  train_acc: {epoch_acc:.4f}  train_f1: {epoch_f1:.4f}")

        # Validation
        if val_loader is not None:
            vloss, vacc, vprecision, vrecall, vauc, vf1 = evaluate_metrics(model, val_loader, criterion, device)
            history["val_loss"].append(vloss)
            history["val_acc"].append(vacc)
            history["val_auc"].append(vauc)
            history["val_f1"].append(vf1)
            history["val_precision"].append(vprecision)
            history["val_recall"].append(vrecall)


            # log per-epoch metrics to mlflow if available
            if 'mlflow' in globals():
                try:
                    mlflow.log_metric("train_loss", epoch_loss, step=epoch)
                    mlflow.log_metric("train_f1", epoch_f1, step=epoch)
                    mlflow.log_metric("train_precision", epoch_precision, step=epoch)
                    mlflow.log_metric("train_recall", epoch_recall, step=epoch)
                    mlflow.log_metric("train_acc", epoch_acc, step=epoch)
                    mlflow.log_metric("train_auc", auc, step=epoch)

                    mlflow.log_metric("val_loss", vloss, step=epoch)
                    mlflow.log_metric("val_f1", vf1, step=epoch)
                    mlflow.log_metric("val_precision", vprecision, step=epoch)
                    mlflow.log_metric("val_recall", vrecall, step=epoch)
                    mlflow.log_metric("val_acc", vacc, step=epoch)
                    mlflow.log_metric("val_auc", vauc, step=epoch)
                except Exception:
                    pass

            tqdm.write(
                f"Epoch {epoch}  train_loss {epoch_loss:.4f} | val_loss {vloss:.4f} | "
                f"val_f1 {vf1:.4f} | val_precision {vprecision:.4f} | val_recall {vrecall:.4f} | val_auc {vauc:.4f}"
            )
        else:
            # keep consistent history lengths
            history["val_loss"].append(None)
            history["val_auc"].append(None)
            history["val_acc"].append(None)
            history["val_f1"].append(None)
            history["val_precision"].append(None)
            history["val_recall"].append(None)

    return history

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import random

BATCH_SIZE = 32

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)


def run_experiments(model_fn,
                    optimizer_fn, criterion, device, epochs, num_classes,
                    seeds=10):

  results = {"acc": [], "precision": [], "recall": [], "auc": [], "loss": []}
  print("run exp")
  for seed in range(seeds):
      print(f"\n=== Seed {seed} ===")
      set_seed(seed)

      all_indices = list(range(len(ds)))
      all_labels = [data_list[i][1] for i in all_indices]  # label_idx for each sample

      trainval_indices, test_indices = train_test_split(
          all_indices,
          test_size=0.1,
          random_state=seed,
          stratify=all_labels  # ← stratify by class labels
      )

      # Extract labels for the trainval subset
      trainval_labels = [all_labels[i] for i in trainval_indices]

      train_indices, val_indices = train_test_split(
          trainval_indices,
          test_size=0.1,
          random_state=seed,
          stratify=trainval_labels  # ← stratify by class labels
      )

      # -------------------------
      # Create Subsets
      # -------------------------
      train_ds = Subset(ds, train_indices)
      val_ds   = Subset(ds, val_indices)
      test_ds  = Subset(ds, test_indices)

      train_loader = make_loader(train_ds, batch_size=BATCH_SIZE)
      val_loader   = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
      test_loader  = make_loader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

      print("Train distribution:", get_label_distribution(train_indices))
      print("Val distribution:", get_label_distribution(val_indices))
      print("Test distribution:", get_label_distribution(test_indices))

      model = model_fn().to(device)
      optimizer = optimizer_fn(model)
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

      # Train
      with mlflow.start_run(nested=True, run_name=f"HAN-CINC-SEED-{seed}"):
        _ = train(model, train_loader, val_loader, optimizer, criterion, device, epochs, scheduler)

        # Evaluate on test set
        loss, acc, precision, recall, auc, f1 = evaluate_metrics(model, test_loader, criterion, device)

        print(f"Test (seed {seed}) — loss: {loss:.4f}, acc: {acc:.4f}, "
            f"precision: {precision:.4f}, recall: {recall:.4f}, auc: {auc:.4f}, f1: {f1:.4f}")

        mlflow.log_metric("test_f1", f1)
        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_acc", acc)
        mlflow.log_metric("test_auc", auc)

      results["loss"].append(loss)
      results["acc"].append(acc)
      results["precision"].append(precision)
      results["recall"].append(recall)
      results["auc"].append(auc)

  # Aggregate results
  print("\n=== Final Results (across seeds) ===")
  for k, v in results.items():
      arr = np.array(v, dtype=np.float32)
      print(f"{k}: mean={arr.mean():.4f}, std={arr.std():.4f}")

  return results


In [ ]:
def model_fn():
    return HANWithAttention(num_classes=num_classes)

def optimizer_fn(model):
    return torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

criterion = nn.CrossEntropyLoss()

results = run_experiments(
    model_fn=model_fn,
    optimizer_fn=optimizer_fn,
    criterion=criterion,
    device=DEVICE,
    epochs=30,
    num_classes=num_classes,
    seeds=10
)

run exp

=== Seed 0 ===
Train distribution: Counter({7: 7363, 10: 1299, 1: 879, 4: 610, 5: 436, 6: 369, 0: 220, 8: 185, 2: 175, 3: 129, 9: 103})
Val distribution: Counter({7: 818, 10: 144, 1: 98, 4: 68, 5: 48, 6: 41, 0: 25, 8: 21, 2: 20, 3: 14, 9: 11})
Test distribution: Counter({7: 909, 10: 160, 1: 108, 4: 75, 5: 54, 6: 46, 0: 27, 8: 23, 2: 22, 3: 16, 9: 13})


Epoch 1/30  train_loss: 1.4212  train_acc: 0.6391  train_f1: 0.5157
Epoch 1  train_loss 1.4212 | val_loss 1.0956 | val_f1 0.5725 | val_precision 0.5428 | val_recall 0.6904 | val_auc 0.6925


Epoch 2/30  train_loss: 1.0225  train_acc: 0.7374  train_f1: 0.6586
Epoch 2  train_loss 1.0225 | val_loss 0.9130 | val_f1 0.6865 | val_precision 0.6461 | val_recall 0.7646 | val_auc 0.7719


Epoch 3/30  train_loss: 0.8949  train_acc: 0.7636  train_f1: 0.6970
Epoch 3  train_loss 0.8949 | val_loss 0.8564 | val_f1 0.7099 | val_precision 0.6578 | val_recall 0.7725 | val_auc 0.8214


Epoch 4/30  train_loss: 0.8309  train_acc: 0.7776  train_f1: 0.7139
Epoch 4  train_loss 0.8309 | val_loss 0.8631 | val_f1 0.7104 | val_precision 0.6658 | val_recall 0.7812 | val_auc 0.8245


...

Epoch 26/30  train_loss: 0.5102  train_acc: 0.8526  train_f1: 0.8246
Epoch 26  train_loss 0.5102 | val_loss 0.4820 | val_f1 0.8334 | val_precision 0.8114 | val_recall 0.8609 | val_auc 0.9175


Epoch 27/30  train_loss: 0.5067  train_acc: 0.8528  train_f1: 0.8247
Epoch 27  train_loss 0.5067 | val_loss 0.4822 | val_f1 0.8356 | val_precision 0.8139 | val_recall 0.8639 | val_auc 0.9159


Epoch 28/30  train_loss: 0.5062  train_acc: 0.8536  train_f1: 0.8254
Epoch 28  train_loss 0.5062 | val_loss 0.4817 | val_f1 0.8330 | val_precision 0.8143 | val_recall 0.8589 | val_auc 0.9179


Epoch 29/30  train_loss: 0.5047  train_acc: 0.8540  train_f1: 0.8268
Epoch 29  train_loss 0.5047 | val_loss 0.4810 | val_f1 0.8364 | val_precision 0.8175 | val_recall 0.8629 | val_auc 0.9173


Epoch 30/30  train_loss: 0.5037  train_acc: 0.8542  train_f1: 0.8266
Epoch 30  train_loss 0.5037 | val_loss 0.4814 | val_f1 0.8330 | val_precision 0.8143 | val_recall 0.8589 | val_auc 0.9183
Test (seed 9) — loss: 0.5204, acc: 0.8464, precision: 0.7999, recall: 0.8464, auc: 0.9112, f1: 0.8130
🏃 View run HAN-CINC-SEED-9 at: https://dbc-5cba9d5d-ddf8.cloud.databricks.com/ml/experiments/3223963235727852/runs/4b145085ca054ac281347c0e4cd81a8a
🧪 View experiment at: https://dbc-5cba9d5d-ddf8.cloud.databricks.com/ml/experiments/3223963235727852

=== Final Results (across seeds) ===
acc: mean=0.8478, std=0.0057
precision: mean=0.8018, std=0.0053
recall: mean=0.8478, std=0.0057
auc: mean=0.9037, std=0.0071
loss: mean=0.5334, std=0.0169


In [ ]:
!pip install calflops -q

In [ ]:
import calflops

In [ ]:
model = HANWithAttention(num_classes=11)

flops, macs, params = calflops.calculate_flops(model=model,
                                      input_shape=(1, 10, 300, 12),
                                      output_as_string=True,
                                      output_precision=4)
print("Model FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  1.23 M  
fwd MACs:                                                               313.337 MMACs
fwd FLOPs:                                                              1.8214 GFLOPS
fwd+bwd MACs:                                                           940.012 MMACs
fwd+bwd FLOPs:                                                          5.4643 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each modu

In [ ]:
!pip install thop -q

In [ ]:
from thop import profile, clever_format

In [ ]:
def analyze_model_complexity(model, input_size=(1, 10, 300, 12), device='cuda'):
    """
    Analyze model complexity: parameters, FLOPs, memory

    Args:
        model: Your HANWithAttention model
        input_size: (batch, segments, timesteps, channels)
        device: 'cuda' or 'cpu'
    """
    print("="*80)
    print("MODEL COMPLEXITY ANALYSIS")
    print("="*80)

    model = model.to(device)
    model.eval()

    # 1. Parameter count
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"\n📊 Parameter Count:")
    print(f"   Total Parameters:       {total_params:,}")
    print(f"   Trainable Parameters:   {trainable_params:,}")
    print(f"   Non-trainable Parameters: {non_trainable_params:,}")
    print(f"   Model Size (MB):        {total_params * 4 / (1024**2):.2f}")  # 4 bytes per float32

    # 2. FLOPs calculation
    dummy_input = torch.randn(input_size).to(device)

    try:
        macs, params = profile(model, inputs=(dummy_input,), verbose=False)
        macs, params = clever_format([macs, params], "%.3f")
        print(f"\nFLOPs (Floating Point Operations):")
        print(f"   MACs (Multiply-Accumulate): {macs}")
        print(f"   Parameters (thop):          {params}")
    except Exception as e:
        print(f"\n FLOPs calculation failed: {e}")

    # 3. Layer-wise parameter breakdown
    print(f"\n Layer-wise Parameter Breakdown:")
    print(f"{'Layer Name':<40} {'Parameters':>15} {'% of Total':>12}")
    print("-"*70)

    for name, param in model.named_parameters():
        if param.requires_grad:
            param_count = param.numel()
            percentage = 100 * param_count / trainable_params
            print(f"{name:<40} {param_count:>15,} {percentage:>11.2f}%")

    # 4. Detailed model summary
    print(f"\nDetailed Model Architecture:")
    summary(model,
            input_size=input_size,
            col_names=["input_size", "output_size", "num_params", "mult_adds"],
            depth=4,
            device=device,
            verbose=0)

    return {
        'total_params': total_params,
        'trainable_params': trainable_params,
        'model_size_mb': total_params * 4 / (1024**2)
    }

analyze_model_complexity(model)

MODEL COMPLEXITY ANALYSIS

📊 Parameter Count:
   Total Parameters:       1,233,691
   Trainable Parameters:   1,233,691
   Non-trainable Parameters: 0
   Model Size (MB):        4.71

🔢 FLOPs (Floating Point Operations):
   MACs (Multiply-Accumulate): 815.630M
   Parameters (thop):          1.233M

📋 Layer-wise Parameter Breakdown:
Layer Name                                    Parameters   % of Total
----------------------------------------------------------------------
conv1d.weight                                     38,400        3.11%
conv1d.bias                                          128        0.01%
channel_attention.mlp.0.weight                     2,048        0.17%
channel_attention.mlp.0.bias                          16        0.00%
channel_attention.mlp.2.weight                     2,048        0.17%
channel_attention.mlp.2.bias                         128        0.01%
lstm_segment.weight_ih_l0                        131,072       10.62%
lstm_segment.weight_hh_l0          

{'total_params': 1233691,
 'trainable_params': 1233691,
 'model_size_mb': 4.706157684326172}

In [ ]:
# From: https://discuss.pytorch.org/t/finding-model-size/130275

param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))


model size: 4.706MB


In [ ]:
import time
import numpy as np
import torch

# =============================================================================
# Benchmark Environment Setup
# =============================================================================

def setup_benchmark_env():
    """
    Freeze backend behavior for reproducible benchmarking.
    """
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

def time_forward(model, inputs, device='cuda'):
    """
    Time a single forward pass with proper synchronization.
    Returns elapsed time in seconds.
    """
    if device == 'cuda':
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        _ = model(inputs)
        end.record()

        torch.cuda.synchronize()
        return start.elapsed_time(end) / 1000.0  # seconds
    else:
        t0 = time.time()
        _ = model(inputs)
        return time.time() - t0


# =============================================================================
# Batch-1 Latency Benchmark
# =============================================================================

def benchmark_latency(
    model,
    input_shape,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure true per-sample inference latency (batch size = 1).
    """
    model = model.to(device).eval()

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(input_shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(input_shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "mean_ms": times.mean() * 1000,
        "std_ms": times.std() * 1000,
        "median_ms": np.median(times) * 1000,
        "min_ms": times.min() * 1000,
        "max_ms": times.max() * 1000,
    }


# =============================================================================
# Batch-N Throughput Benchmark
# =============================================================================

def benchmark_throughput(
    model,
    input_shape,
    batch_size,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure throughput under batched inference.
    """
    model = model.to(device).eval()
    shape = (batch_size,) + input_shape[1:]

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "batch_time_ms": times.mean() * 1000,
        "per_sample_ms": (times.mean() / batch_size) * 1000,
        "samples_per_sec": batch_size / times.mean()
    }

def run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cuda'
):
    setup_benchmark_env()

    print("=" * 80)
    print("INFERENCE BENCHMARK")
    print("=" * 80)
    print(f"Device: {device}")
    print(f"Input shape: {input_shape}")
    print(f"Throughput batch size: {batch_size}")

    latency = benchmark_latency(
        model=model,
        input_shape=input_shape,
        device=device
    )

    throughput = benchmark_throughput(
        model=model,
        input_shape=input_shape,
        batch_size=batch_size,
        device=device
    )

    print("\nBatch-1 Latency (ms):")
    for k, v in latency.items():
        print(f"  {k:>10}: {v:.3f}")

    print("\nBatch-N Throughput:")
    for k, v in throughput.items():
        if "ms" in k:
            print(f"  {k:>16}: {v:.3f}")
        else:
            print(f"  {k:>16}: {v:.2f}")

    return {
        "latency": latency,
        "throughput": throughput
    }

results = run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cpu'
)

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


INFERENCE BENCHMARK
Device: cpu
Input shape: (1, 10, 300, 12)
Throughput batch size: 16

Batch-1 Latency (ms):
     mean_ms: 22.414
      std_ms: 0.328
   median_ms: 22.380
      min_ms: 21.644
      max_ms: 24.295

Batch-N Throughput:
     batch_time_ms: 392.339
     per_sample_ms: 24.521
   samples_per_sec: 40.78


In [ ]:

results = run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cuda'
)

INFERENCE BENCHMARK
Device: cuda
Input shape: (1, 10, 300, 12)
Throughput batch size: 16

Batch-1 Latency (ms):
     mean_ms: 3.025
      std_ms: 0.031
   median_ms: 3.020
      min_ms: 2.989
      max_ms: 3.202

Batch-N Throughput:
     batch_time_ms: 6.691
     per_sample_ms: 0.418
   samples_per_sec: 2391.15


In [ ]:
import psutil
import tracemalloc
# ============================================================================
# 3. MEMORY USAGE TRACKING
# ============================================================================


def measure_memory_usage(model, test_loader, device='cuda'):
    """
    Measure GPU/CPU memory usage during inference.

    Returns:
        dict with memory usage info (peak memory, model memory, etc.)
    """
    print("\n" + "="*80)
    print("MEMORY USAGE ANALYSIS")
    print("="*80)

    model.eval().to(device)

    sample_batch = next(iter(test_loader))
    inputs = sample_batch["signal"].to(device)

    if device == 'cuda':
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

        # Allocate model (parameters) memory
        model_memory_mb = sum(p.numel() * 4 for p in model.parameters()) / (1024**2)  # float32
        print(f"\n💾 GPU Model Memory (parameters only): {model_memory_mb:.2f} MB")

        # Forward pass and measure peak memory
        with torch.no_grad():
            _ = model(inputs)

        peak_memory_mb = torch.cuda.max_memory_allocated() / (1024**2)
        print(f"💾 Peak GPU memory during inference: {peak_memory_mb:.2f} MB")

        # GPU info
        if torch.cuda.is_available():
            props = torch.cuda.get_device_properties(0)
            total_memory_gb = props.total_memory / (1024**3)
            gpu_name = torch.cuda.get_device_name(0)
            print(f"\n🖥️ GPU: {gpu_name}, Total Memory: {total_memory_gb:.2f} GB")
            print(f"💡 Memory utilization: {peak_memory_mb / (total_memory_gb * 1024):.2%}")

        return {
            "model_memory_mb": model_memory_mb,
            "peak_memory_mb": peak_memory_mb
        }

    else:  # CPU
        process = psutil.Process()
        mem_before = process.memory_info().rss / (1024**2)

        # Optional: tracemalloc for more precise Python/Tensor allocations
        tracemalloc.start()

        with torch.no_grad():
            _ = model(inputs)

        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        mem_after = process.memory_info().rss / (1024**2)

        print(f"\n💾 CPU Memory Usage:")
        print(f"   RSS before inference: {mem_before:.2f} MB")
        print(f"   RSS after inference:  {mem_after:.2f} MB")
        print(f"   RSS increase:         {mem_after - mem_before:.2f} MB")
        print(f"   tracemalloc peak:     {peak / 1024**2:.2f} MB")

        # System info
        virtual_mem = psutil.virtual_memory()
        print(f"\n🖥️ System RAM: Total {virtual_mem.total/1024**3:.2f} GB, Available {virtual_mem.available/1024**3:.2f} GB, Usage {virtual_mem.percent}%")

        return {
            "rss_before_mb": mem_before,
            "rss_after_mb": mem_after,
            "rss_increase_mb": mem_after - mem_before,
            "tracemalloc_peak_mb": peak / 1024**2
        }

measure_memory_usage(model, test_loader, "cuda")



MEMORY USAGE ANALYSIS

💾 GPU Model Memory (parameters only): 4.71 MB
💾 Peak GPU memory during inference: 265.21 MB

🖥️ GPU: NVIDIA L4, Total Memory: 22.16 GB
💡 Memory utilization: 1.17%


{'model_memory_mb': 4.706157684326172, 'peak_memory_mb': 265.21044921875}

In [ ]:
measure_memory_usage(model, test_loader, "cpu")


MEMORY USAGE ANALYSIS

💾 CPU Memory Usage:
   RSS before inference: 5026.20 MB
   RSS after inference:  5026.20 MB
   RSS increase:         0.00 MB
   tracemalloc peak:     0.00 MB

🖥️ System RAM: Total 52.96 GB, Available 46.39 GB, Usage 12.4%


{'rss_before_mb': 5026.203125,
 'rss_after_mb': 5026.203125,
 'rss_increase_mb': 0.0,
 'tracemalloc_peak_mb': 0.002590179443359375}